In [7]:
#Generación Archivo
#importar librerias
import pandas as pd
import random as rd
import string as st
from datetime import datetime
from datetime import timedelta
from google.cloud.exceptions import NotFound

#Para conexión a GCP - Storage
import google.cloud.storage as storage
import os


In [8]:
path_specs = '../data/TP_Final_Especificacion_1.xlsx'
path_coord = '../data/ar.csv'
path_cred = '../usm-infra-grupo1-42c8c1c4178a.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_cred

In [9]:
# Leer las hojas de cálculo del archivo
df_sheets_desc = pd.read_excel(path_specs, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']
coordenadas = pd.read_csv(path_coord)

In [10]:
def generar_codigos_sku(productos):
  codigos_sku = {}
  for producto in productos:
    caracteres = st.ascii_uppercase + st.digits
    sku_codigo = ''.join(rd.choice(caracteres) for _ in range(10))
    codigos_sku[producto] = sku_codigo

  return codigos_sku

In [11]:
def generate_fecha_cierre(fecha_cierre: datetime) -> str:
    """
    Genera una fecha de cierre de comprobante aleatoria entre la fecha de cierre
    de comprobante y la fecha actual.
    """
    fech_cie_com = (fecha_cierre - timedelta(hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))) \
            .strftime('%Y-%m-%d %H:%M:%S')
    return fech_cie_com

In [12]:
#generar SKUs a partir de la lista de productos
codigos_sku = generar_codigos_sku(df_productos['nombre'])

#nombre de las columnas de los archivos cliente,venta,stock
columns_v = list(df_sheets_desc['Desc_Ventas']['Campo']) + ['n_distribuidor']
columns_s = list(df_sheets_desc['Desc_Stock']['Campo']) + ['n_distribuidor']
columns_m = list(df_sheets_desc['Desc_Clientes']['Campo']) + ['n_distribuidor']
columns_d = list(df_sheets_desc['Desc_Deuda']['Campo']) + ['n_distribuidor']

#cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 90

#fecha actual
fecha_actual = datetime.now()

#cantidad de distribuidores
cant_dist = 6 # 6 - 1

#cantidad de clientes
cant_clientes = 7

#cantidad sucursales
cant_sucursales = 5

#instancia storage client
storage_client = storage.Client()

for distribuidor in range(1,cant_dist):

  for d in range(0,cant_dias):

    #fecha de cierre
    fech_cierre = fecha_actual - timedelta(days=d)
    fech_cierre_string = fech_cierre.strftime('%Y-%m-%d')
    fech_cie_com_data = [ generate_fecha_cierre(fech_cierre) for _ in range(cant_clientes)]

    #generamos los datos de venta unidades
    vta_unidades_data = [(rd.randint(0,100)) for _ in range(cant_clientes)]

    #generamos los datos de venta en importes
    vta_importe_data = [round((rd.uniform(100,1000)),2) for _ in range(cant_clientes)]

    #generamos la condicion de venta
    cond_vta_data = [ rd.choice(df_cond_vta['codigo_condicion']) for _ in range(cant_clientes)]

    codigo_sucursal_base = (distribuidor * 100)  # Código base para el distribuidor

    #sucursales del distribuidor
    #el dist1 va a tener sucursales de la 26 a 50
    #el dist2 va a tener sucursales de la 51 a 75
    sucursales_dist = [(codigo_sucursal_base + rd.randint(10, 99)) for _ in range(cant_clientes)]

    #clientes de la sucursal del distribuidor
    clientes = [(codigo_sucursal_base + rd.randint(10, 99)) for _ in range(cant_clientes)]

    #Provincia y ciudades
    provincias = [(rd.choice(df_localidades.columns.values)) for _ in range(cant_clientes)]
    ciudades_aleatorias = []
    for _ in range(cant_clientes):
      for p in provincias:
          ciudades_provincia = df_localidades[p].dropna().tolist()
          ciudad_aleatoria = rd.choice(ciudades_provincia)
          ciudades_aleatorias.append(ciudad_aleatoria)

    #Estado
    estado = [(rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]) for _ in range(cant_clientes)]

    #Nombre_ cliente
    nombre_cliente = [(f'Cliente_{_}') for _ in range(cant_clientes)]

    #CUIT
    cuit = [(rd.randint(10000000, 99999999) ) for _ in range(cant_clientes)]

    #Razon Social
    razon_social = [(f"Empresa_{_}") for _ in range(cant_clientes)]

    #Direccion
    direccion = [(f"Dirección_{_}") for _ in range(cant_clientes)]

    #Dias de visita
    dias_visita = [(rd.choice(df_dias_visita['codigo_dia'])) for _ in range(cant_clientes)]

    #Telefono
    telefono = [(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}") for _ in range(cant_clientes)]

    #Fecha de alta y de baja
    fecha_alta = [(fech_cierre - timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d') for _ in range(cant_clientes)]
    fecha_baja = [ (fech_cierre + timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d') if estado[idx] == "I" else pd.NA for idx in range(cant_clientes)] 

    #Coordenadas
    # coordenada_latitud = [(round(rd.uniform(-90, 90), 6)) for _ in range(cant_clientes)]   #agregar coordenada de acuerdo a la ciudad
    # coordenada_longitud = [(round(rd.uniform(-180, 180), 6)) for _ in range(cant_clientes)]#agregar coordenada de acuerdo a la ciudad
    coordenada_latitud = []
    coordenada_longitud = []
    for ciudad in ciudades_aleatorias:
        coordenada_latitud.append(list(coordenadas[coordenadas['city'] == ciudad]['lat'])[0] + rd.random()/100)
        coordenada_longitud.append(list(coordenadas[coordenadas['city'] == ciudad]['lng'])[0]+ rd.random()/100)

    #Tipo de negocio
    tipo_negocio = [(rd.choice(df_tipo_negocio['nombre'])) for _ in range(cant_clientes)]

    #ARCHIVO STOCK
    #Unidad
    unidad = [(rd.choice(df_unidades['codigo_unidad'])) for _ in range(cant_clientes)]
    #Stock
    stock = [(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100)) for _ in range(cant_clientes)]
    #Producto
    producto=[]
    producto1=[]
    producto1 = list(codigos_sku.keys())
    producto1.extend(producto1)
    producto=producto1
    #SKU
    sku_codigo=[]
    sku_codigo1=[]
    sku_codigo1 = list(codigos_sku.values())
    sku_codigo1.extend(sku_codigo1)
    sku_codigo = sku_codigo1

    #DEUDA
    deuda_vencida = [(round(rd.uniform(0, 1000), 2)) for _ in range(cant_clientes)]
    deuda_total = [(round(rd.uniform(0, 10000), 2)) for _ in range(cant_clientes)]

    # Columna n_distribuidor
    n_distribuidor = [distribuidor for _ in range(cant_clientes)]

    #data frame con data
    venta_clientes_data = list(zip(sucursales_dist,clientes,fech_cie_com_data,sku_codigo,vta_unidades_data,vta_importe_data,cond_vta_data,n_distribuidor))
    stock_data = list(zip(sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad, n_distribuidor))
    clientes_data = list(zip(sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio, n_distribuidor))
    deuda_data = list(zip(sucursales_dist,clientes, fech_cie_com_data,deuda_vencida,deuda_total , n_distribuidor))

    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    #upload file
    #Creamos Buckets
    bucket_name = 'g1_distribuidor_{}'.format(distribuidor)

    try:
    # Verificar si el bucket ya existe antes de crearlo
      bucket = storage_client.get_bucket(bucket_name)
    except NotFound:
    #Si no existe, crear el bucket
      bucket = storage_client.create_bucket(bucket_name)

    #guardamos archivos

    #Rutas de los archivos en el bucket
    archivo_destino_v = 'venta/{}.csv'.format(fech_cierre_string)
    archivo_destino_s = 'stock/{}.csv'.format(fech_cierre_string)
    archivo_destino_c = 'cliente/{}.csv'.format(fech_cierre_string)
    archivo_destino_d = 'deuda/{}.csv'.format(fech_cierre_string)

    #Subir los archivos al bucket
    bucket.blob(archivo_destino_v).upload_from_string(df_vta_cli.to_csv(index=False), content_type='text/csv')
    bucket.blob(archivo_destino_s).upload_from_string(df_stock.to_csv(index=False), content_type='text/csv')
    bucket.blob(archivo_destino_c).upload_from_string(df_clientes.to_csv(index=False), content_type='text/csv')
    bucket.blob(archivo_destino_d).upload_from_string(df_deuda.to_csv(index=False), content_type='text/csv')

    #Cerrar el cliente de almacenamiento
    storage_client.close()

    #upload_file('distribuidor_'+str(distribuidor), archivo_destino_v, df_vta_cli)
    #upload_file('distribuidor_'+str(distribuidor), archivo_destino_s, df_stock)
    #upload_file('distribuidor_'+str(distribuidor), archivo_destino_m, df_maestro)

  #df_vta_cli.to_csv(archivo_destino_v, encoding='utf-8', index=False)
  #df_stock.to_csv(archivo_destino_s, encoding='utf-8', index=False)
  #df_maestro.to_csv(archivo_destino_m, encoding='utf-8', index=False)

